In [28]:
%%writefile app_Dashboard.py 
#####################################################
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.figure_factory as ff

######################################################
#Definimos la instancia
@st.cache_resource
######################################################
#Creamos la función de carga de datos
def load_data():
    df = pd.read_csv("Valencia_España.csv")
    df1 = pd.read_csv("México_sin_nulos.csv")
    df2 = pd.read_csv("Datos_Limpios_de_bangkook.csv")
    df3 = pd.read_csv("Albany.csv")

    #Concierte a Dolares Valencia
    conversion_rate = 1.08  # ejemplo: 1 euro = 1.08 dólares
    df['price_usd'] = df['price'] * conversion_rate
    df = df[df['price'] <= 800]
    #Dolares Mexico
    tipo_cambio = 17.00
    df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
    df1['price_usd'] = df1['price'] / tipo_cambio

    # Dólares Bangkook
    cambio = 34.00
    df2['price'] = pd.to_numeric(df2['price'], errors='coerce')
    df2['price_usd'] = df2['price'] / cambio

    # Selecciono columnas relevantes
    selected_cols = [
        'host_id', 'host_name', 'neighbourhood', 'host_is_superhost',
        'room_type', 'accommodates', 'amenities', 'price', 'minimum_nights',
        'number_of_reviews', 'reviews_per_month', 'host_identity_verified',
        'availability_365', 'host_response_time', 'bathrooms_text', 'bathrooms',
        'bedrooms', 'beds', 'has_availability', 'instant_bookable'
    ]
    df = df[selected_cols]
    df1 = df1[selected_cols]
    df2 = df2[selected_cols]
    df3 = df3[selected_cols]
    # Columnas numéricas
    numeric_df = df.select_dtypes(include=['float', 'int'])
    numeric_cols = numeric_df.columns

    # Columnas de texto/categóricas
    text_df = df.select_dtypes(include=['object'])
    text_cols = text_df.columns

    # Categorías únicas de 'room_type'
    unique_categories_room = df['room_type'].unique()

    return df, df1, df2, df3, numeric_cols, text_cols, unique_categories_room, numeric_df

###############################################################################
#Cargo los datos obtenidos de la función "load_data"
df, df1, df2, df3, numeric_cols, text_cols, unique_categories_room, numeric_df = load_data()
###############################################################################
View = st.sidebar.selectbox(
    label="Selecciona una sección:", 
    options=["Análisis Categórico y Numérico"]
)
#####################################################################################
if View == "Análisis Categórico y Numérico":
    st.title("Exploración de Categorías y Variables Numéricas")

    # Añadir pestañas para mejor organización
    vista = st.sidebar.selectbox("Tipo de análisis", ["Categórico", "Numérico"])

    if vista == "Categórico":
        st.write("Contenido categórico")
        
        st.header("Distribución Categórica por Ciudad")

        city_options = {
            "Valencia": df,
            "México": df1,
            "Bangkok": df2,
            "Albany": df3
        }

        selected_cat_col = st.sidebar.selectbox("Selecciona una variable categórica", text_cols)
        n_top = st.sidebar.slider("Mostrar top categorías", 3, 15, 5)

        for city, city_df in city_options.items():
            if st.sidebar.checkbox(f"Mostrar gráfico para {city}", key=f"{city}_cat"):
                st.subheader(f"{city}")
                if selected_cat_col in city_df.columns:
                    freq_table = city_df[selected_cat_col].value_counts().reset_index()
                    freq_table.columns = [selected_cat_col, "Frecuencia"]
                    total = freq_table["Frecuencia"].sum()
                    freq_table["Porcentaje"] = (freq_table["Frecuencia"] / total * 100).round(2)
                    freq_table_top = freq_table.sort_values(by="Frecuencia", ascending=False).head(n_top)

                    col1, col2 = st.columns(2)
                    with col1:
                        fig_pie = px.pie(
                            freq_table_top, 
                            names=selected_cat_col, 
                            values="Frecuencia", 
                            title=f"{city}: Distribución de {selected_cat_col}"
                        )
                        fig_pie.update_traces(textposition='inside', textinfo='percent+label')
                        st.plotly_chart(fig_pie, use_container_width=True)

                    with col2:
                        fig_bar = px.bar(
                            freq_table_top, 
                            x=selected_cat_col, 
                            y="Frecuencia",
                            color="Frecuencia",
                            title=f"{city}: Frecuencia de {selected_cat_col}"
                        )
                        st.plotly_chart(fig_bar, use_container_width=True)

                    if st.checkbox(f"Mostrar tabla de frecuencias para {city}", key=f"{city}_tabla"):
                        st.dataframe(freq_table.style.background_gradient(cmap="Blues", subset=["Frecuencia"]))

    elif vista == "Numérico":
        st.write("Contenido numérico")
        st.header("Distribución Numérica por Ciudad")
        city_options = {
            "Valencia": df,
            "México": df1,
            "Bangkok": df2,
            "Albany": df3
        }
        numerics_vars_selected = st.sidebar.multiselect(
            label="Variables numéricas a graficar", options=numeric_cols
        )
        category_selected = st.sidebar.selectbox("Filtrar por tipo de habitación", options=unique_categories_room)
        plot_type = st.sidebar.radio("Tipo de gráfico", ["Line Plot", "Scatter Plot"])
        show_trend = st.sidebar.checkbox("Mostrar línea de tendencia (solo Scatter)", value=True)
        st.sidebar.markdown("----")
        for city, city_df in city_options.items():
            if st.sidebar.checkbox(f"Mostrar gráfico para {city}", key=f"{city}_num"):
                st.subheader(f"{city}")
                data = city_df[city_df['room_type'] == category_selected]

                if numerics_vars_selected:
                    data_features = data[numerics_vars_selected]
                    
                    if plot_type == "Line Plot":
                        fig_line = px.line(
                            data_frame=data_features,
                            x=data_features.index,
                            y=numerics_vars_selected,
                            title=f"{city}: Variables numéricas para {category_selected}",
                            height=500,
                        )
                        fig_line.update_layout(hovermode="x unified")
                        st.plotly_chart(fig_line, use_container_width=True)

                    elif plot_type == "Scatter Plot" and len(numerics_vars_selected) >= 2:
                        fig_scatter = px.scatter(
                            data_frame=data_features, 
                            x=numerics_vars_selected[0], 
                            y=numerics_vars_selected[1],
                            title=f"{city}: Relación entre {numerics_vars_selected[0]} y {numerics_vars_selected[1]}",
                            opacity=0.7,
                            size_max=10,
                            trendline="ols" if show_trend else None
                        )
                        st.plotly_chart(fig_scatter, use_container_width=True)
                else:
                    st.info(f"Selecciona variables numéricas para mostrar datos de {city}.")

Overwriting app_Dashboard.py
